In [ ]:
#Import Necessary Packages
import os
import bs4
import streamlit as st
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import AIMessage ,HumanMessage ,SystemMessage
import fitz
from gtts import gTTS

os.environ["OPENAI_API_KEY"] = "Your_API_KEY"

In [ ]:
def load_pdf_file_content(file_path):
    text_content = ''
    with fitz.open(file_path) as doc:
        for page in doc:
            text_content += page.get_text()
    return text_content

# Simple document class
class SimpleDocument:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata or {}

# Loader class for documents
class TextFileLoader:
    def __init__(self, file_paths):
        self.file_paths = file_paths

    def load(self):
        docs = []
        for file_path in self.file_paths:
            text_content = load_pdf_file_content(file_path)
            docs.append(SimpleDocument(page_content=text_content))
        return docs

# Initial setup: Load PDF, process content
loader = TextFileLoader(file_paths=("Azure_AI.pdf",))  # Update path
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

class LangChainRAGChatbot:
    def __init__(self):
        self.retriever = vectorstore.as_retriever()
        self.prompt = hub.pull("rlm/rag-prompt")
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
        self.previous_context=""

    def ask_question(self, question):
        # Combine previous context with the new question for input
        combined_input = f"{self.previous_context}\nQ: {question}\nA:" if self.previous_context else f"Q: {question}\nA:"

        # Adjust the call to `invoke` to use the expected `input` parameter
        result = self.llm.invoke(input=combined_input, max_tokens=150)

        # Process the result as needed (assuming result is directly the answer text)
        # Update the context with the new question and answer
        self.previous_context += f"\nQ: {question}\nA: {result}"

        return result

    def format_docs(self,docs):
      return '\n\n'.join(doc.page_content for doc in docs)

    def format_context(self,previous_context,new_question):
      return f"{previous_context}\nQ: {new_question}\nA:"

    def update_context(self,question,answer):
      self.previous_context += f"\nQ: {question}\nA: {answer}"

# Function to generate and save audio file using gTTS
def text_to_speech(text, language, save_path='output.mp3'):
    # gTTS for text-to-speech
    tts = gTTS(text, lang=language, slow=False)
    tts.save(save_path)
    return save_path

In [ ]:
chatbot=LangChainRAGChatbot()

In [ ]:
# Streamlit web application

def main():
  st.title("Multilingual Storytelling and Text-to-Speech App")

  # User input
  text_input=st.text_area("Enter the text for Multilingual Storytelling")
  target_language=st.selectbox("Select target language:", ["es","fr","de","it"])

  # Storytelling button
  if st.button('Multilingual Storytelling'):
    if text_input:
      story_telling=chatbot.ask_question(text_input)
      st.subheader("Multilingual Storytelling")
      st.write(story_telling)

     # Text-to-Speech button
      if st.button('Text-to-Speech'):
        audio_file_path= "output_audio.mp3"
        text_to_speech(story_telling,target_language,audio_file_path)
        st.audio(audio_file_path,format="audio/mp3",start_time=0)

        # Remove the audio file after playing
        os.remove(audio_file_path)

if __name__=='__main__':
  main()


